In [1]:
from time import sleep
import time
from confluent_kafka import Producer
import json
import random
from datetime import datetime, timedelta

In [2]:
# Kafka broker address on AWS
MODE = 'local'

if MODE == 'local':
    bootstrap_servers = 'localhost:9092'
else:
    bootstrap_servers = ['10.110.245.59:9092']  # Replace with your AWS Kafka broker endpoints

# Kafka topic to publish data to
topic = 'JSON_ingestion'

# Initialize Kafka producer
producer = Producer({'bootstrap.servers': bootstrap_servers})

def delivery_report(err, msg):
    if err is not None:
        print(f'Message delivery failed: {err}')
    else:
        print(f'Message delivered to {msg.topic()} [{msg.partition()}]')

In [3]:
def progress_bar(duration):
    total_time = duration  # Total time for the progress bar to complete (in seconds)
    start_time = time.time()

    while time.time() - start_time < duration:
        elapsed_time = time.time() - start_time
        progress = elapsed_time / total_time
        progress_bar_length = 40
        completed_length = int(progress * progress_bar_length)
        remaining_length = progress_bar_length - completed_length

        bar = '[' + '=' * completed_length + ' ' * remaining_length + ']'
        percent_complete = int(progress * 100)
        time_remaining = int((total_time - elapsed_time) + 1)  # Adding 1 to prevent negative time remaining

        print(f'\rProgress: {bar} {percent_complete}% - Time remaining: {time_remaining}s', end='', flush=True)
        time.sleep(0.1)  # Adjust sleep time to control the speed of the progress bar

    print("\nTask completed!")

In [4]:
# Function to generate random data
websites = [
    "https://www.google.com",
    "https://www.youtube.com",
    "https://www.facebook.com",
    "https://www.amazon.com",
    "https://www.yahoo.com",
    "https://www.wikipedia.org",
    "https://www.twitter.com",
    "https://www.instagram.com",
    "https://www.linkedin.com",
    "https://www.reddit.com",
    "https://www.netflix.com",
    "https://www.ebay.com",
    "https://www.microsoft.com",
    "https://www.apple.com",
    "https://www.imdb.com",
    "https://www.twitch.tv",
    "https://www.spotify.com",
    "https://www.paypal.com",
    "https://www.bing.com",
    "https://www.aliexpress.com",
    "https://www.pinterest.com",
    "https://www.booking.com",
    "https://www.cnn.com",
    "https://www.bbc.com",
    "https://www.weather.com",
    "https://www.forbes.com",
    "https://www.nytimes.com",
    "https://www.stackoverflow.com",
    "https://www.github.com",
    "https://www.udemy.com",
    "https://www.medium.com",
    "https://www.quora.com",
    "https://www.walmart.com",
    "https://www.target.com",
    "https://www.bestbuy.com",
    "https://www.homedepot.com",
    "https://www.lowes.com",
    "https://www.dell.com",
    "https://www.hp.com",
    "https://www.ibm.com",
    "https://www.oracle.com",
    "https://www.nike.com",
    "https://www.adidas.com",
    "https://www.ikea.com",
    "https://www.macys.com",
    "https://www.bloomberg.com",
    "https://www.reuters.com",
    "https://www.cnet.com",
    "https://www.engadget.com",
    "https://www.techcrunch.com"
]

countries = [
    "Afghanistan", "Albania", "Algeria", "Andorra", "Angola", "Antigua and Barbuda", "Argentina", "Armenia",
    "Australia", "Austria", "Azerbaijan", "Bahamas", "Bahrain", "Bangladesh", "Barbados", "Belarus", "Belgium",
    "Belize", "Benin", "Bhutan", "Bolivia", "Bosnia and Herzegovina", "Botswana", "Brazil", "Brunei", "Bulgaria",
    "Burkina Faso", "Burundi", "Cabo Verde", "Cambodia", "Cameroon", "Canada", "Central African Republic", "Chad",
    "Chile", "China", "Colombia", "Comoros", "Congo", "Costa Rica", "Croatia", "Cuba", "Cyprus", "Czech Republic",
    "Democratic Republic of the Congo", "Denmark", "Djibouti", "Dominica", "Dominican Republic", "East Timor",
    "Ecuador", "Egypt", "El Salvador", "Equatorial Guinea", "Eritrea", "Estonia", "Eswatini", "Ethiopia", "Fiji",
    "Finland", "France", "Gabon", "Gambia", "Georgia", "Germany", "Ghana", "Greece", "Grenada", "Guatemala",
    "Guinea", "Guinea-Bissau", "Guyana", "Haiti", "Honduras", "Hungary", "Iceland", "India", "Indonesia", "Iran",
    "Iraq", "Ireland", "Israel", "Italy", "Ivory Coast", "Jamaica", "Japan", "Jordan", "Kazakhstan", "Kenya",
    "Kiribati", "Kosovo", "Kuwait", "Kyrgyzstan", "Laos", "Latvia", "Lebanon", "Lesotho", "Liberia", "Libya",
    "Liechtenstein", "Lithuania", "Luxembourg", "Madagascar", "Malawi", "Malaysia", "Maldives", "Mali", "Malta",
    "Marshall Islands", "Mauritania", "Mauritius", "Mexico", "Micronesia", "Moldova", "Monaco", "Mongolia",
    "Montenegro", "Morocco", "Mozambique", "Myanmar", "Namibia", "Nauru", "Nepal", "Netherlands", "New Zealand",
    "Nicaragua", "Niger", "Nigeria", "North Korea", "North Macedonia", "Norway", "Oman", "Pakistan", "Palau",
    "Palestine", "Panama", "Papua New Guinea", "Paraguay", "Peru", "Philippines", "Poland", "Portugal", "Qatar",
    "Romania", "Russia", "Rwanda", "Saint Kitts and Nevis", "Saint Lucia", "Saint Vincent and the Grenadines",
    "Samoa", "San Marino", "Sao Tome and Principe", "Saudi Arabia", "Senegal", "Serbia", "Seychelles", "Sierra Leone",
    "Singapore", "Slovakia", "Slovenia", "Solomon Islands", "Somalia", "South Africa", "South Korea", "South Sudan",
    "Spain", "Sri Lanka", "Sudan", "Suriname", "Sweden", "Switzerland", "Syria", "Taiwan", "Tajikistan", "Tanzania",
    "Thailand", "Togo", "Tonga", "Trinidad and Tobago", "Tunisia", "Turkey", "Turkmenistan", "Tuvalu", "Uganda",
    "Ukraine", "United Arab Emirates", "United Kingdom", "United States", "Uruguay", "Uzbekistan", "Vanuatu",
    "Vatican City", "Venezuela", "Vietnam", "Yemen", "Zambia", "Zimbabwe"
]

def generate_data():
    ip = str(random.randint(1, 256)) + '.' + \
     str(random.randint(1, 256)) + '.' + \
     str(random.randint(1, 256)) + '.' + \
     str(random.randint(1, 256))
    
    start_date = datetime(2020, 1, 1)
    end_date = datetime.now()
    start_timestamp = start_date.timestamp()
    end_timestamp = end_date.timestamp()
    #random_timestamp = random.uniform(start_timestamp, end_timestamp)
    random_timestamp = datetime.fromtimestamp(random.uniform(start_timestamp, end_timestamp))

    session_end = random_timestamp + timedelta(seconds=random.randint(1, 256))
    
    data = {
        "ip_address": ip,
        "website": random.choice(websites),
        "location": random.choice(countries),
        "session_start": str(random_timestamp),
        "session_end": str(session_end),
        "no_of_clicks": random.randint(0, 20)
    }
    
    return data

# Main function to continuously generate and publish data
def main():
    bucket = ""
    for j in range(50000):
        for i in range(200):
            data = generate_data()
            bucket = bucket + json.dumps(data) + '\n'
            #producer.send(topic, value=data)
        sleep(0.1) # Wait for 1 second before generating next data
        #print(bucket)
        producer.produce(topic, bucket, callback=delivery_report)
        bucket = ""
        producer.flush()  # Wait for all messages to be delivered
    
if __name__ == "__main__":
    main()

Message delivered to JSON_ingestion [0]
Message delivered to JSON_ingestion [0]
Message delivered to JSON_ingestion [0]
Message delivered to JSON_ingestion [0]
Message delivered to JSON_ingestion [0]
Message delivered to JSON_ingestion [0]
Message delivered to JSON_ingestion [0]
Message delivered to JSON_ingestion [0]
Message delivered to JSON_ingestion [0]
Message delivered to JSON_ingestion [0]
Message delivered to JSON_ingestion [0]
Message delivered to JSON_ingestion [0]
Message delivered to JSON_ingestion [0]
Message delivered to JSON_ingestion [0]
Message delivered to JSON_ingestion [0]
Message delivered to JSON_ingestion [0]
Message delivered to JSON_ingestion [0]
Message delivered to JSON_ingestion [0]
Message delivered to JSON_ingestion [0]
Message delivered to JSON_ingestion [0]
Message delivered to JSON_ingestion [0]
Message delivered to JSON_ingestion [0]
Message delivered to JSON_ingestion [0]
Message delivered to JSON_ingestion [0]
Message delivered to JSON_ingestion [0]


KeyboardInterrupt: 